Vision Transformer Model is imported from [ViT](https://www.kaggle.com/abhinand05/vit-base-models-pretrained-pytorch)
For details on ViT, visit my blog : [ViT For Begiiners](https://dev.to/rohitgupta24/vision-transformer-an-image-is-worth-1616-words-3o2g)

In [ ]:
!pip install wtfml==0.0.2
!pip install timm


**Do upvote if this notebook helps you.**
**I will be using tez library from Abhishek Thakur**

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image

from sklearn import model_selection
from sklearn import metrics

import timm

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import albumentations as A

from wtfml.utils import EarlyStopping
from wtfml.engine import Engine
from wtfml.data_loaders.image import ClassificationLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Read the data by loading all the input files and create folds.
train_dir = '../input/happy-whale-and-dolphin/train_images'
test_dir = '../input/happy-whale-and-dolphin/test_images'
df = pd.read_csv("../input/testcsv/demo.csv")
df.head()

In [ ]:
# checking the number of files in train_dir and csv
t = os.listdir(train_dir)
y = len(df.species),len(t)

#number of species in csv file
x =len(pd.unique(df.species))
print(x,y)

In [ ]:
df.species.value_counts()

In [ ]:
#Data Cleaning
#Found out some duplicate labels

print("Number of unique species before fixing : ", df['species'].nunique())

df['species'].replace({
    'bottlenose_dolpin' : 'bottlenose_dolphin',
    'kiler_whale' : 'killer_whale',
},inplace =True)

print("Number of unique species after fixing : ", df['species'].nunique())


#df['class'] = df['species'].apply(lambda x: x.split('_')[-1])
df.head()

In [ ]:
'''visualizing  the data count'''

#visualization imports
#to have better idea of eda, please visit : https://www.kaggle.com/awsaf49/happywhale-data-distribution#Dolphin-Species-6%EF%B8%8F%E2%83%A3

import plotly.express as px

data = df.species.value_counts().reset_index()
fig = px.bar(data, x='index', y='species', color='species',title='Species Count')
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()



In [ ]:
#Cross Validation #imbalanced dataset so stratifi

'''
# import pandas and model_selection module of scikit-learn
import pandas as pd
from sklearn import model_selection
if __name__ == "__main__":
    # Training data is in a csv file called train.csv
    df = pd.read_csv("train.csv")
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # the next step is to randomize the rows of the data
    df = df.sample(frac=1).reset_index(drop=True)
    # fetch targets
    y = df.target.values
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=5)
    # fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f
# save the new csv with kfold column
df.to_csv("train_folds.csv", index=False)

'''
#Not trying the above Stratified K Fold for now. Instead will be using train_test split from model_selection to make validation and training set

dfx = pd.read_csv('../input/testcsv/demo.csv')
df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=42, stratify=dfx.species.values
)
len(df_train),len(df_valid)

# so we have not created folds inthe input, only divided the data into training and validation set.
df_train.head()

In [ ]:
#reset the index and than drop the index
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_train.shape

In [ ]:
#join image name with path to make a list of training images

train_images = [os.path.join(train_dir,x) for x in df_train.image.values]
train_images[1]


In [ ]:
valid_images = [os.path.join(train_dir,x) for x in df_valid.image.values]
valid_images[:5]

In [ ]:
train_targets = df_train.species.values
valid_targets = df_valid.species.values

train_targets[1]

In [ ]:
!pip install tez
import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

In [ ]:
import albumentations

train_aug = albumentations.Compose([
            albumentations.RandomResizedCrop(256, 256),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)], p=1.)
  
        
valid_aug = albumentations.Compose([
            albumentations.CenterCrop(256, 256, p=1.),
            albumentations.Resize(256, 256),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

print("hello")